In [14]:
import math
import logging
from datetime import datetime


import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [15]:
datasets = load_dataset("klue", "sts")
testsets = load_dataset("kor_nlu", "sts")
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)
    


Found cached dataset klue (C:/Users/maili/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset kor_nlu (C:/Users/maili/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
model_save_path = "training_klue_sts_klue-roberta-base-2022-12-19_04-33-15"
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)
# print(test_evaluator(model, output_path=model_save_path))


# from konlpy.tag import Okt
# voc = '심근경색 전조증상이 의심됩니다 주위에서 골든타임 특히 조심해야 한다고 중간중간 체크하라고 하네요 특히 가슴 통증하고 만성피로가 느껴진다고 했었거든요'
# okt_pos = Okt().pos(voc, norm=True, stem=True)
# print(okt_pos)



docs = [
        "12345",
        "15432"
]
document_embeddings = model.encode(docs)

query = "12345"
query_embedding = model.encode(query)


top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    if score > 0.2:
        print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

입력 문장: 12345

<입력 문장과 유사한 2 개의 문장>

1: 12345 (유사도: 1.0000)

2: 15432 (유사도: 0.4535)



In [9]:

from konlpy.tag import Okt
voc = '가슴을 쥐어짜는 거 같아요 숨을 쉬기 힘들어요'
okt_pos = Okt().pos(voc, norm=True, stem=True)

list1 = []
list1 = []
for i in range(len(okt_pos)):
    if okt_pos[i][1] == "Noun":
        list1.append(okt_pos[i][0])
    elif okt_pos[i][1] == "Verb":
        list1.append(okt_pos[i][0])
    elif okt_pos[i][1] == "Adjective":
        list1.append(okt_pos[i][0])

str =" ".join(list1)
print(str)


가슴 쥐다 짜다 거 같다 숨 쉬다 힘들다


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
      "심근경색 전조증상이 의심됩니다 주위에서 골든타임 특히 조심해야 한다고 중간중간 체크하라고 하네요 특히 가슴 통증하고 만성피로가 느껴진다고 했었거든요",
        "왼쪽 가슴이 시린 느낌 싸한 느낌이라고 할지 서늘하다고 할지 모를 느낌입니다 눌러서 아픈 곳은 없으나 가슴이 답답하고 이 불쾌감 때문에 안절부절못하게 됩니다 가끔가다 명치부터 올라오는 싸한 느낌 혹은 놀이 기구 등을 탈 때의 얼얼한 느낌이 있습니다"
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
#print(vector.vocabulary_) #




import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(A, B):
    return dot(A, B)/(norm(A)*norm(B))

k = vector.fit_transform(corpus).toarray()
cosine_similarity(k[0], k[1])

[[0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1
  1 1 1 1 2 1 1 0 1 0]
 [1 0 2 0 1 1 1 1 0 2 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 2 1 1 1 1 1 1 0 1 0 0
  0 0 0 0 0 0 0 2 0 1]]


0.0